In [ ]:

import datetime
# FOR FUN PURPOSES - DOES NOT PREDICT ANYTHING


In [ ]:

def forecast_stock(stock,start=datetime.datetime(2010, 1, 1),end=datetime.date.today()):
    import pandas as pd
    import datetime
    import pandas_datareader.data as web
    from pandas import Series, DataFrame
    
    import math
    import numpy as np
    import sklearn
    from sklearn import preprocessing
    from sklearn.linear_model import LinearRegression
    from sklearn.neighbors import KNeighborsRegressor

    from sklearn.linear_model import Ridge
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline
    from sklearn.model_selection import train_test_split
    %matplotlib inline
    import matplotlib.pyplot as plt
    from matplotlib import style

    # Adjusting the size of matplotlib
    import matplotlib as mpl
    mpl.rc('figure', figsize=(8, 7))
    mpl.__version__
    
    df = web.DataReader(stock, 'yahoo', start, end)
    
    #dfreg = df.loc[:,[‘Adj Close’,’Volume’]]
    dfreg = df.loc[:,['Adj Close','Volume']]
    dfreg['HL_PCT'] = (df['High'] - df['Low']) / df['Close'] * 100.0
    dfreg['PCT_change'] = (df['Close'] - df['Open']) / df['Open'] * 100.0
    

    # Drop missing value
    dfreg.fillna(value=-99999, inplace=True)
    # We want to separate 1 percent of the data to forecast
    forecast_out = int(math.ceil(0.01 * len(dfreg)))
    # Separating the label here, we want to predict the AdjClose
    forecast_col = 'Adj Close'
    dfreg['label'] = dfreg[forecast_col].shift(-forecast_out)
    X = np.array(dfreg.drop(['label'], 1))
    # Scale the X so that everyone can have the same distribution for linear regression
    X = preprocessing.scale(X)
    # Finally We want to find Data Series of late X and early X (train) for model generation and evaluation
    X_lately = X[-forecast_out:]
    X = X[:-forecast_out]
    # Separate label and identify it as y
    y = np.array(dfreg['label'])
    y = y[:-forecast_out]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Linear regression
    clfreg = LinearRegression(n_jobs=-1)
    clfreg.fit(X_train, y_train)
    # Quadratic Regression 2
    clfpoly2 = make_pipeline(PolynomialFeatures(2), Ridge())
    clfpoly2.fit(X_train, y_train)

    # Quadratic Regression 3
    clfpoly3 = make_pipeline(PolynomialFeatures(4), Ridge())
    clfpoly3.fit(X_train, y_train)
    
    # KNN Regression
    clfknn = KNeighborsRegressor(n_neighbors=4)
    clfknn.fit(X_train, y_train)
    
    confidencereg = clfreg.score(X_test, y_test)
    confidencepoly2 = clfpoly2.score(X_test,y_test)
    confidencepoly3 = clfpoly3.score(X_test,y_test)
    confidenceknn = clfknn.score(X_test, y_test)
    forecast_set = clfpoly3.predict(X_lately)
    dfreg['Forecast'] = ''
    dfreg['Forecast'] = np.nan
    print ("confidencereg:",confidencereg,"confidencepoly2:",confidencepoly2,"confidencepoly3:",confidencepoly3,"confidenceknn:",confidenceknn)
    
    
    last_date = dfreg.iloc[-1].name
    last_unix = last_date
    next_unix = last_unix + datetime.timedelta(days=1)

    for i in forecast_set:
        next_date = next_unix
        next_unix += datetime.timedelta(days=1)
        dfreg.loc[next_date] = [np.nan for _ in range(len(dfreg.columns)-1)]+[i]
    dfreg['Adj Close'].tail(250).plot()
    dfreg['Forecast'].tail(250).plot()
    plt.legend(loc=4)
    plt.xlabel('Date')
    plt.ylabel(stock + ' Price')
    plt.show()
    

In [ ]:


import timeit

all_stock = ['ALUA.BA', 'APBR.BA', 'BBAR.BA', 'BMA.BA', 'BYMA.BA', 'CEPU.BA', 'COME.BA', 'CRES.BA', 'CVH.BA', 'DGCU2.BA', 'EDN.BA', 'GGAL.BA', 'LOMA.BA', 'METR.BA', 'MIRG.BA', 'PAMP.BA', 'SUPV.BA', 'TECO2.BA',  'TGSU2.BA', 'TRAN.BA', 'TXAR.BA', 'VALO.BA', 'YPFD.BA']
#start = datetime.datetime(2010, 1, 1)
#end = datetime.datetime(2010, 1, 1)
for stock in all_stock:
    start = timeit.timeit()
    print("starting forecast",stock)

    #############################################################
    #### KNN PREDICTION AFTER END DATE ( START2END IS TRAINING )
    #############################################################
    forecast_stock(stock,start=datetime.datetime(2010, 1, 1),
                   end=datetime.datetime(2019, 11, 10))
   
    end = timeit.timeit()
    print("forecast time : ",end - start)

